In [1]:
import os
os.chdir("../")
%pwd

'/Users/mukulagarwal/Desktop/Projects/Text-Summarizer'

In [3]:
from src.textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path:Path
    tokenizer_name:str

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        
        return data_transformation_config

In [10]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'],truncation = True, max_length = 1024)
        target_encodings = self.tokenizer(example_batch['summary'],truncation = True, max_length = 128)
        
        input_encodings['labels'] = target_encodings['input_ids']
        
        return input_encodings
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt =  dataset_samsum.map(
            self.convert_examples_to_features,batched=True
        )
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [11]:
config_manager = ConfigurationManager()
data_transformation_config = config_manager.get_data_transformation_config()
data_transformation = DataTransformation(data_transformation_config)
data_transformation.convert()

[2025-04-13 18:43:13,762: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-13 18:43:13,765: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-13 18:43:13,766: INFO: common: created directory at: artifacts]
[2025-04-13 18:43:13,767: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 304296.29 examples/s]
